## Step 1: Create spark context

In [ ]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
spark

In [ ]:
sc

## Step 2: Create a pandas dataframe

In [ ]:
import pandas as pd
p_df = pd.read_csv('data/users.csv', sep='|')

## Step 3: Create spark dataframe

In [ ]:
rdd = sc.textFile("data/users.csv").map(lambda line: line.split("|"))

In [ ]:
rdd.take(5)

In [ ]:
df = spark.read.csv("data/users.csv", sep="|")

In [ ]:
type(df)

In [ ]:
# it's already a DF, but this is the easy way to rename columns
df = (spark.read.csv("data/users.csv", sep="|")
           .toDF("id", "age", "gender", "occupation", "zip"))

In [ ]:
(
    df.where("occupation != 'other'")
      .groupby("occupation")
      .count()
      .sort("count", ascending=0)
      .show()
)

In [ ]:
df.persist()

In [ ]:
df.limit(5).toPandas()
#df.head()
#df.show(5)

In [ ]:
def show(df, n=5):
    return df.limit(n).toPandas()

In [ ]:
import pyspark.sql.functions as F
df.agg(F.countDistinct('occupation')).show()

In [ ]:
query = """
SELECT occupation, COUNT(*) as count
FROM users
GROUP BY occupation
ORDER BY count DESC
"""
df.createOrReplaceTempView('users')
output = spark.sql(query)
output.show(10)